In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import html
import csv
import time
from tqdm import tqdm 


Sometime the web scraping code shows "500 Server Error: Internal Server Error for url:". It is because nature of my loop definition. I ask the date from 1 to 31 for each month in each year, but sometimes there is no 31 days within a month, so it reports an error. 

For instance, ""1981年2月31日异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/31/4/"", and ""1981年4月31日异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/04/31/1/"". 

Because there is just no 31st in April or February.


# data from 1946-05-15 to 2003-12-31


There are only small amount of changes in code afterwards. Becasue I web-scraped from Chinese website, I used Chinese to write annotation for the code. Just for you could read I re-write the annotation of **15/5/1946 - 31/5/1946** part in English. The following code and annotation are writen in same logic, so I did not re-write them into English version.

1. get text from 15/5/1946 -- 31/5/1946:

In [ ]:
# basement settings
base_template = "https://cn.govopendata.com/renminribao/1946/5/{date}/{page}/"
csv_filename = 'historical_news.csv'
request_interval = 1  # request interval in seconds
pages_to_crawl = 4    # set {page} from 1 to 4

# Create a session with camouflage
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(date, page):
    try:
        # Generate canonical URL
        url = base_template.format(
            date=str(date).zfill(2),  # Date formatted with two digits
            page=page
        )
        
        # Initiate a request
        response = session.get(url)
        response.encoding = 'utf-8'  # for Chinese characters
        
        # Skip pages that don't exist (when status_code is 404)
        if response.status_code == 404:
            return False
        response.raise_for_status()
        
        # Parsing structured data
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # Data cleaning process
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # Content extraction
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("page:")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)  # 移除HTML标签
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # write to CSV
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([
                1946,           # year
                5,              # month
                date,           # day
                page,           # page_number
                processed_content
            ])
            
        return True

    except Exception as e:
        print(f"{date}May, {page} is abnormal: {str(e)}")
        return False

if __name__ == "__main__":
    # intitialize CSV file
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # create a date range for May 15 to 31
    date_range = range(15, 32)
    
    # Design progress bar
    with tqdm(total=len(date_range)*pages_to_crawl, desc="overall process") as pbar:
        for date in date_range:
            success_count = 0
            for page in range(1, pages_to_crawl+1):
                if process_page(date, page):
                    success_count += 1
                pbar.update(1)
                time.sleep(request_interval)
            print(f"{date} May: already get{success_count}/{pages_to_crawl} page")
    
    print(f"data saves to {csv_filename}")

# The rest of codeing just expands the text range step by step.

2. get text 1/6/1946 -- 30/12/1946:

In [ ]:

base_template = "https://cn.govopendata.com/renminribao/1946/{month}/{date}/{page}/"
csv_filename = '1946_full_year_news.csv'
request_interval = 1  # 请求间隔(秒)
pages_to_crawl = 4    # 每日期待抓取页数

# 创建带伪装的会话
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(month, date, page):
    """处理指定月份和日期的单个版面"""
    try:
        # 生成规范URL（月份和日期都补零）
        url = base_template.format(
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        # 发起请求
        response = session.get(url)
        response.encoding = 'utf-8'
        
        # 跳过不存在页面
        if response.status_code == 404:
            return False
        response.raise_for_status()
        
        # 解析结构化数据
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # 数据清洗流程
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入CSV
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([
                1946,           # year
                month,          # month
                date,           # day
                page,           # page_number
                processed_content
            ])
            
        return True

    except Exception as e:
        print(f"{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV文件
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 时间范围配置
    months_range = range(6, 13)  # 6-12月
    days_range = range(1, 32)    # 每月尝试抓取1-31日
    
    # 计算总任务量
    total_tasks = len(months_range) * len(days_range) * pages_to_crawl
    
    # 带进度条的执行
    with tqdm(total=total_tasks, desc="抓取进度") as pbar:
        for month in months_range:
            for date in days_range:
                success_count = 0
                for page in range(1, pages_to_crawl + 1):
                    if process_page(month, date, page):
                        success_count += 1
                    pbar.update(1)
                    time.sleep(request_interval)
                print(f"{month}月{date}日: 成功抓取{success_count}/{pages_to_crawl}版")
    
    print(f"全年数据已存储至 {csv_filename}")

3. 1947 whole year:

In [ ]:

# 修改年份和文件名
base_template = "https://cn.govopendata.com/renminribao/1947/{month}/{date}/{page}/"
csv_filename = '1947_full_year_news.csv'  # 文件名包含年份
request_interval = 1
pages_to_crawl = 4

# 会话配置保持不变
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(month, date, page):
    """新增年份参数传递逻辑"""
    try:
        url = base_template.format(
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()
        
        # 数据解析逻辑保持不变
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # 数据清洗流程
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 修改年份参数为1947
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([
                1947,           # 修改为1947年
                month,
                date,
                page,
                processed_content
            ])
        return True

    except Exception as e:
        print(f"{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 扩展月份范围至全年
    months_range = range(1, 13)  # 1-12月
    days_range = range(1, 32)
    
    # 进度条优化
    with tqdm(total=len(months_range), desc="月份进度", unit="月", ncols=80) as month_pbar:
        for month in months_range:
            month_pbar.set_description(f"当前月份 {month}月")
            monthly_success = 0
            
            for date in days_range:
                for page in range(1, pages_to_crawl + 1):
                    if process_page(month, date, page):
                        monthly_success += 1
                    time.sleep(request_interval)
            
            # 显示当月统计
            month_pbar.set_postfix_str(f"完成页面: {monthly_success}/{len(days_range)*pages_to_crawl}")
            month_pbar.update(1)
    
    print(f"1947全年数据已存储至 {csv_filename}")

4. 1948 to 1950

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import html
import csv
import time
from tqdm import tqdm

# 新增年份参数的URL模板
base_template = "https://cn.govopendata.com/renminribao/{year}/{month}/{date}/{page}/"
csv_filename = '1948-1950_news_archive.csv'  # 统一存储文件名
request_interval = 1
pages_to_crawl = 4

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(year, month, date, page):  # 新增year参数
    try:
        url = base_template.format(
            year=year,
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()

        # 保持原有解析逻辑不变
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入当前年份数据
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([
                year,  # 动态年份值
                month,
                date,
                page,
                processed_content
            ])
        return True

    except Exception as e:
        print(f"{year}年{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV文件
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 新增年份范围参数
    years_range = range(1948, 1951)  # 包含1950
    months_range = range(1, 13)
    days_range = range(1, 32)
    
    # 年份进度条优化
    with tqdm(total=len(years_range), desc="年份进度", unit="年", ncols=80) as year_pbar:
        for year in years_range:
            year_pbar.set_description(f"当前年份 {year}")
            annual_success = 0
            
            for month in months_range:
                for date in days_range:
                    for page in range(1, pages_to_crawl + 1):
                        if process_page(year, month, date, page):
                            annual_success += 1
                        time.sleep(request_interval)
            
            # 显示年度统计
            year_pbar.set_postfix_str(f"完成请求: {annual_success}/{12 * 31 * 4}")
            year_pbar.update(1)

    print(f"1948-1950年数据已存储至 {csv_filename}")

5. 1951 to 1970

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import html
import csv
import time
from tqdm import tqdm

# 更新年份范围和文件名
base_template = "https://cn.govopendata.com/renminribao/{year}/{month}/{date}/{page}/"
csv_filename = '1951-1970_news_archive.csv'  # 更新文件名
request_interval = 1
pages_to_crawl = 4

# 保持会话配置不变
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(year, month, date, page):
    """处理页面逻辑保持不变"""
    try:
        url = base_template.format(
            year=year,
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()

        # 保持解析逻辑
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入数据
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([year, month, date, page, processed_content])
        return True

    except Exception as e:
        print(f"{year}年{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 更新年份范围（1951-1970含两端）
    years_range = range(1951, 1971)  # 生成1951-1970的年份序列
    months_range = range(1, 13)
    days_range = range(1, 32)
    
    # 进度条优化（年份级别）
    with tqdm(total=len(years_range), desc="抓取进度", unit="年", ncols=100) as pbar:
        for year in years_range:
            pbar.set_description(f"当前年份 {year}")
            annual_count = 0
            
            # 每月处理
            for month in months_range:
                for date in days_range:
                    for page in range(1, pages_to_crawl + 1):
                        if process_page(year, month, date, page):
                            annual_count += 1
                        time.sleep(request_interval)
            
            # 更新进度
            pbar.set_postfix_str(f"成功请求 {annual_count}/1488")  # 1488 = 12 * 31 * 4
            pbar.update(1)

    print(f"1951-1970年新闻数据已存储至 {csv_filename}")

6. 1971 to 1980

In [ ]:

# 更新年份范围和文件名
base_template = "https://cn.govopendata.com/renminribao/{year}/{month}/{date}/{page}/"
csv_filename = '1981-1991_news_archive.csv'  # 更新文件名
request_interval = 1
pages_to_crawl = 4

# 保持会话配置不变
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(year, month, date, page):
    """处理页面逻辑保持不变"""
    try:
        url = base_template.format(
            year=year,
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()

        # 保持解析逻辑
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # 数据清洗流程
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入数据
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([year, month, date, page, processed_content])
        return True

    except Exception as e:
        print(f"{year}年{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 更新年份范围（1971-1990含两端）
    years_range = range(1971, 1982)  # 1992保证包含1991
    months_range = range(1, 13)
    days_range = range(1, 32)
    
    # 进度条优化（年份级别）
    with tqdm(total=len(years_range), desc="抓取进度", unit="年", ncols=100) as pbar:
        for year in years_range:
            pbar.set_description(f"当前年份 {year}")
            annual_count = 0
            
            # 每月处理
            for month in months_range:
                for date in days_range:
                    for page in range(1, pages_to_crawl + 1):
                        if process_page(year, month, date, page):
                            annual_count += 1
                        time.sleep(request_interval)
            
            # 更新进度
            pbar.set_postfix_str(f"成功请求 {annual_count}")  #  = 11 * 31 * 4
            pbar.update(1)

    print(f"1981-1991年新闻数据已存储至 {csv_filename}")

7. 1981 to 1991

In [4]:

# 更新年份范围和文件名
base_template = "https://cn.govopendata.com/renminribao/{year}/{month}/{date}/{page}/"
csv_filename = '1981-1991_news_archive.csv'  # 更新文件名
request_interval = 1
pages_to_crawl = 4

# 保持会话配置不变
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(year, month, date, page):
    """处理页面逻辑保持不变"""
    try:
        url = base_template.format(
            year=year,
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()

        # 保持解析逻辑
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # 数据清洗流程
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入数据
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([year, month, date, page, processed_content])
        return True

    except Exception as e:
        print(f"{year}年{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 更新年份范围（1971-1990含两端）
    years_range = range(1981, 1992)  # 1992保证包含1991
    months_range = range(1, 13)
    days_range = range(1, 32)
    
    # 进度条优化（年份级别）
    with tqdm(total=len(years_range), desc="抓取进度", unit="年", ncols=100) as pbar:
        for year in years_range:
            pbar.set_description(f"当前年份 {year}")
            annual_count = 0
            
            # 每月处理
            for month in months_range:
                for date in days_range:
                    for page in range(1, pages_to_crawl + 1):
                        if process_page(year, month, date, page):
                            annual_count += 1
                        time.sleep(request_interval)
            
            # 更新进度
            pbar.set_postfix_str(f"成功请求 {annual_count}")  #  = 11 * 31 * 4
            pbar.update(1)

    print(f"1981-1991年新闻数据已存储至 {csv_filename}")

当前年份 1981:   0%|                                                         | 0/11 [00:00<?, ?年/s]

1981年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/29/1/
1981年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/29/2/
1981年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/29/3/
1981年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/29/4/
1981年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/30/1/
1981年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/30/2/
1981年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/30/3/
1981年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1981/02/30/4/
1981年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 1982:   9%|██▋                           | 1/11 [44:00<7:20:02, 2640.25s/年, 成功请求 1459]

1982年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/29/1/
1982年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/29/2/
1982年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/29/3/
1982年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/29/4/
1982年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/30/1/
1982年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/30/2/
1982年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/30/3/
1982年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1982/02/30/4/
1982年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 1983:  18%|█████                       | 2/11 [1:29:35<6:44:24, 2696.05s/年, 成功请求 1460]

...

1990年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/29/1/
1990年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/29/2/
1990年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/29/3/
1990年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/29/4/
1990年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/30/1/
1990年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/30/2/
1990年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/30/3/
1990年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1990/02/30/4/
1990年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 1991:  91%|██████████████████████████▎  | 10/11 [7:40:43<46:46, 2806.58s/年, 成功请求 1460]

1991年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/29/1/
1991年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/29/2/
1991年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/29/3/
1991年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/29/4/
1991年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/30/1/
1991年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/30/2/
1991年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/30/3/
1991年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1991/02/30/4/
1991年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 1991: 100%|█████████████████████████████| 11/11 [8:27:29<00:00, 2768.10s/年, 成功请求 1460]

1981-1991年新闻数据已存储至 1981-1991_news_archive.csv


7. 1992 to 2003

In [28]:

# 更新年份范围和文件名
base_template = "https://cn.govopendata.com/renminribao/{year}/{month}/{date}/{page}/"
csv_filename = '1992-2003.csv'  # 更新文件名
request_interval = 1
pages_to_crawl = 4

# 保持会话配置不变
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'zh-CN,zh;q=0.9'
})

def process_page(year, month, date, page):
    """处理页面逻辑保持不变"""
    try:
        url = base_template.format(
            year=year,
            month=str(month).zfill(2),
            date=str(date).zfill(2),
            page=page
        )
        
        response = session.get(url)
        response.encoding = 'utf-8'
        
        if response.status_code == 404:
            return False
        response.raise_for_status()

        # 保持解析逻辑
        soup = BeautifulSoup(response.text, 'html.parser')
        script_tag = soup.find('script', {'type': 'application/ld+json'})
        if not script_tag:
            return False
            
        # 数据清洗流程
        cleaned_data = re.sub(r'[\x00-\x1F\x7F]', '', script_tag.string)
        decoded_data = html.unescape(cleaned_data)
        news_article = json.loads(decoded_data)
        
        # 内容处理
        raw_content = news_article.get('articleBody', '')
        content_start = raw_content.find("专栏：")
        processed_content = raw_content[content_start:] if content_start != -1 else raw_content
        processed_content = re.sub(r'<.*?>', '', processed_content)
        processed_content = '\n'.join([line.strip() for line in processed_content.split('\n') if line.strip()])
        
        # 写入数据
        with open(csv_filename, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f)
            writer.writerow([year, month, date, page, processed_content])
        return True

    except Exception as e:
        print(f"{year}年{month}月{date}日第{page}版异常: {str(e)}")
        return False

if __name__ == "__main__":
    # 初始化CSV
    with open(csv_filename, 'w', encoding='utf-8-sig') as f:
        f.write("year,month,date,page,text\n")
    
    # 更新年份范围
    years_range = range(1992, 2003)  # 1992保证包含1991
    months_range = range(1, 13)
    days_range = range(1, 32)
    
    # 进度条优化（年份级别）
    with tqdm(total=len(years_range), desc="抓取进度", unit="年", ncols=100) as pbar:
        for year in years_range:
            pbar.set_description(f"当前年份 {year}")
            annual_count = 0
            
            # 每月处理
            for month in months_range:
                for date in days_range:
                    for page in range(1, pages_to_crawl + 1):
                        if process_page(year, month, date, page):
                            annual_count += 1
                        time.sleep(request_interval)
            
            # 更新进度
            pbar.set_postfix_str(f"成功请求 {annual_count}")  #  
            pbar.update(1)

    print(f"1992-2003年新闻数据已存储至 {csv_filename}")

当前年份 1992:   0%|                                                         | 0/11 [00:00<?, ?年/s]

1992年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/30/1/
1992年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/30/2/
1992年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/30/3/
1992年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/30/4/
1992年2月31日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/31/1/
1992年2月31日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/31/2/
1992年2月31日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/31/3/
1992年2月31日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1992/02/31/4/
1992年3月28日第4版异常: 400 Client Error: Bad Request f

当前年份 1993:   9%|██▋                           | 1/11 [45:42<7:37:02, 2742.29s/年, 成功请求 1463]

1993年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/29/1/
1993年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/29/2/
1993年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/29/3/
1993年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/29/4/
1993年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/30/1/
1993年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/30/2/
1993年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/30/3/
1993年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/1993/02/30/4/
1993年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 1994:  18%|█████                       | 2/11 [1:32:44<6:58:25, 2789.50s/年, 成功请求 1447]

...

2001年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/29/1/
2001年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/29/2/
2001年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/29/3/
2001年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/29/4/
2001年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/30/1/
2001年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/30/2/
2001年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/30/3/
2001年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2001/02/30/4/
2001年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 2002:  91%|██████████████████████████▎  | 10/11 [7:43:47<46:17, 2777.13s/年, 成功请求 1459]

2002年2月29日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/29/1/
2002年2月29日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/29/2/
2002年2月29日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/29/3/
2002年2月29日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/29/4/
2002年2月30日第1版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/30/1/
2002年2月30日第2版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/30/2/
2002年2月30日第3版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/30/3/
2002年2月30日第4版异常: 500 Server Error: Internal Server Error for url: https://cn.govopendata.com/renminribao/2002/02/30/4/
2002年2月31日第1版异常: 500 Server Error: Internal Serv

当前年份 2002: 100%|█████████████████████████████| 11/11 [8:29:35<00:00, 2779.58s/年, 成功请求 1459]

1992-2003年新闻数据已存储至 1992-2003.csv
